In [1]:
import glob, os

In [2]:
os.chdir("0")

In [14]:
files = glob.glob("[0-9]*_Right.csv")

In [15]:
files

['0_Right.csv',
 '10_Right.csv',
 '1_Right.csv',
 '2_Right.csv',
 '3_Right.csv',
 '4_Right.csv',
 '5_Right.csv',
 '6_Right.csv',
 '7_Right.csv',
 '8_Right.csv',
 '9_Right.csv']

In [34]:
import pandas as pd

df_list = []

for file in files:
    df = pd.read_csv(file, index_col=None).drop(['Unnamed: 0'], axis = 1)
    df['Time'] = pd.to_datetime(df['Time'].str[:-3], format = '%H:%M:%S.%f')
#     df = df.diff().iloc[1:]
#     df['Time'] = df['Time'].dt.total_seconds()
#     df.insert(loc = 0, column = 'File', value = file)
    
#     for col in df.iloc[:,2:]:
#         df[col + ' velocity'] = df[col]/df['Time']
    
    df_list.append(df)

In [17]:
len(df_list)

11

In [35]:
df_list[0]

,Time,thumbProximal_L_X,thumbProximal_L_Y,thumbProximal_L_Z,thumbDistal_L_X,thumbDistal_L_Y,thumbDistal_L_Z,thumbEF_L_X,thumbEF_L_Y,thumbEF_L_Z,...,armFrontRadius_L_Z,armFrontUlna_L_X,armFrontUlna_L_Y,armFrontUlna_L_Z,armBackLateral_L_X,armBackLateral_L_Y,armBackLateral_L_Z,armBackMedial_L_X,armBackMedial_L_Y,armBackMedial_L_Z
0,1900-01-01 22:16:06.075,0.152098,-0.069129,0.455693,0.160663,-0.078444,0.480445,0.174084,-0.088726,0.489291,...,0.380085,0.205600,-0.081617,0.387774,0.190003,-0.152103,0.166404,0.226784,-0.162750,0.174093
1,1900-01-01 22:16:06.103,0.151998,-0.069086,0.455394,0.160584,-0.078405,0.480138,0.174015,-0.088680,0.488978,...,0.379765,0.205434,-0.081503,0.387476,0.189919,-0.152496,0.166262,0.226714,-0.163081,0.173974
2,1900-01-01 22:16:06.139,0.151996,-0.068888,0.455207,0.160600,-0.078180,0.479954,0.174030,-0.088452,0.488799,...,0.379530,0.205339,-0.081383,0.387283,0.189960,-0.152772,0.166187,0.226749,-0.163342,0.173940
3,1900-01-01 22:16:06.172,0.151960,-0.068592,0.454957,0.160527,-0.077884,0.479717,0.173924,-0.088173,0.488593,...,0.379279,0.205255,-0.081220,0.387079,0.189834,-0.153253,0.166195,0.226603,-0.163864,0.173994
4,1900-01-01 22:16:06.206,0.151822,-0.067965,0.454742,0.160298,-0.077276,0.479526,0.173614,-0.087658,0.488414,...,0.379066,0.205075,-0.081143,0.387023,0.189835,-0.153883,0.166359,0.226499,-0.164734,0.174316
5,1900-01-01 22:16:06.242,0.151684,-0.067315,0.454716,0.160063,-0.076643,0.479527,0.173335,-0.087086,0.488408,...,0.379064,0.205091,-0.080787,0.387172,0.190410,-0.153731,0.166537,0.227003,-0.164709,0.174645
6,1900-01-01 22:16:06.276,0.151393,-0.066427,0.455002,0.159707,-0.075714,0.479850,0.172933,-0.086233,0.488712,...,0.379247,0.204924,-0.080520,0.387552,0.191159,-0.153024,0.166713,0.227644,-0.164210,0.175018
7,1900-01-01 22:16:06.319,0.150745,-0.065504,0.455062,0.158840,-0.074829,0.479968,0.171938,-0.085495,0.488845,...,0.379310,0.204588,-0.080455,0.387808,0.191189,-0.152923,0.166935,0.227511,-0.164492,0.175433
8,1900-01-01 22:16:06.354,0.150152,-0.065236,0.455117,0.158124,-0.074550,0.480066,0.171191,-0.085246,0.488951,...,0.379403,0.204359,-0.080458,0.387985,0.191392,-0.152682,0.167006,0.227669,-0.164332,0.175588
9,1900-01-01 22:16:06.391,0.149094,-0.064614,0.455295,0.156794,-0.073975,0.480312,0.169699,-0.084805,0.489272,...,0.379566,0.203682,-0.080561,0.388389,0.191587,-0.152340,0.167216,0.227684,-0.164361,0.176039


In [30]:
right_frame = pd.concat(df_list, axis=0, ignore_index=False)

In [31]:
right_frame

,File,Time,thumbProximal_L_X,thumbProximal_L_Y,thumbProximal_L_Z,thumbDistal_L_X,thumbDistal_L_Y,thumbDistal_L_Z,thumbEF_L_X,thumbEF_L_Y,...,armFrontRadius_L_Z velocity,armFrontUlna_L_X velocity,armFrontUlna_L_Y velocity,armFrontUlna_L_Z velocity,armBackLateral_L_X velocity,armBackLateral_L_Y velocity,armBackLateral_L_Z velocity,armBackMedial_L_X velocity,armBackMedial_L_Y velocity,armBackMedial_L_Z velocity
1,0_Right.csv,0.028,-0.000100,0.000042,-0.000299,-0.000078,0.000039,-3.071000e-04,-0.000069,0.000047,...,-0.011446,-0.005929,0.004074,-0.010636,-0.002986,-0.014021,-0.005057,-0.002518,-0.011814,-0.004243
2,0_Right.csv,0.036,-0.000002,0.000199,-0.000187,0.000015,0.000225,-1.838000e-04,0.000015,0.000228,...,-0.006517,-0.002631,0.003348,-0.005369,0.001111,-0.007658,-0.002089,0.000983,-0.007264,-0.000942
3,0_Right.csv,0.033,-0.000035,0.000296,-0.000250,-0.000073,0.000296,-2.370000e-04,-0.000106,0.000279,...,-0.007594,-0.002558,0.004939,-0.006185,-0.003788,-0.014573,0.000233,-0.004445,-0.015824,0.001639
4,0_Right.csv,0.034,-0.000138,0.000626,-0.000216,-0.000229,0.000609,-1.911000e-04,-0.000309,0.000515,...,-0.006279,-0.005271,0.002269,-0.001644,0.000003,-0.018544,0.004815,-0.003047,-0.025574,0.009450
5,0_Right.csv,0.036,-0.000138,0.000650,-0.000026,-0.000236,0.000633,7.000000e-07,-0.000279,0.000571,...,-0.000039,0.000436,0.009889,0.004153,0.015972,0.004231,0.004967,0.013997,0.000686,0.009161
6,0_Right.csv,0.034,-0.000291,0.000888,0.000287,-0.000356,0.000928,3.237000e-04,-0.000403,0.000853,...,0.005362,-0.004906,0.007838,0.011165,0.022026,0.020791,0.005162,0.018865,0.014674,0.010965
7,0_Right.csv,0.043,-0.000648,0.000924,0.000060,-0.000867,0.000885,1.173000e-04,-0.000995,0.000738,...,0.001463,-0.007821,0.001527,0.005953,0.000719,0.002342,0.005160,-0.003098,-0.006553,0.009649
8,0_Right.csv,0.035,-0.000592,0.000268,0.000055,-0.000716,0.000279,9.870000e-05,-0.000747,0.000249,...,0.002674,-0.006529,-0.000098,0.005054,0.005797,0.006874,0.002034,0.004503,0.004563,0.004411
9,0_Right.csv,0.037,-0.001059,0.000622,0.000178,-0.001330,0.000575,2.456000e-04,-0.001492,0.000441,...,0.004389,-0.018297,-0.002789,0.010927,0.005251,0.009243,0.005673,0.000403,-0.000768,0.012214
10,0_Right.csv,0.040,-0.002325,0.001945,-0.000007,-0.003050,0.001846,1.670000e-04,-0.003674,0.001414,...,-0.003675,-0.050437,-0.018241,0.017105,0.014225,0.003450,0.006965,-0.003450,-0.031142,0.027747


In [32]:
right_frame.groupby('File')[' thumbProximal_L_X'].agg('mean')

File
0_Right.csv    -0.000113
10_Right.csv    0.000134
1_Right.csv    -0.000006
2_Right.csv    -0.000012
3_Right.csv    -0.000003
4_Right.csv     0.000037
5_Right.csv     0.000020
6_Right.csv    -0.000051
7_Right.csv     0.000093
8_Right.csv    -0.000068
9_Right.csv    -0.000071
Name:  thumbProximal_L_X, dtype: float64

In [11]:
right_frame.groupby('File')[' thumbProximal_L_Y'].agg('mean')

File
0_Right.csv     0.000042
10_Right.csv   -0.000151
1_Right.csv     0.000038
2_Right.csv     0.000007
3_Right.csv     0.000023
4_Right.csv     0.000044
5_Right.csv     0.000062
6_Right.csv    -0.000139
7_Right.csv     0.000004
8_Right.csv    -0.000072
9_Right.csv    -0.000066
Name:  thumbProximal_L_Y, dtype: float64